# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import requests

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City Name,Latitude,Longitude,Temperature,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Kattivākkam,13.2167,80.3167,82.31,83.10,80.69,88,100,0.78,IN
1,1,Qaqortoq,60.7167,-46.0333,30.74,30.74,30.74,97,100,10.76,GL
2,2,Cole Harbour,44.6724,-63.4751,16.93,17.92,14.83,80,0,1.01,CA
3,3,Waitangi,-43.9535,-176.5597,56.37,56.37,56.37,89,80,18.34,NZ
4,4,Dudinka,69.4058,86.1778,-12.42,-12.42,-12.42,100,100,7.67,RU


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
cityPlot = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 896,
    frame_height = 504,
    hover_cols = ["City Name", "Temperature", "Country"]
)

cityPlot
    

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City Name,Humidity,Temperature,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
# max temp: 90 F
# min temp: ~ 50 F
# wind < 10
#cloudiness = < 50

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] >70) & (city_data_df["Max Temp"] < 90)\
    & (city_data_df["Min Temp"] >50)
    & (city_data_df["Humidity"] <70)
    & (city_data_df["Wind Speed"] < 10) \
    & (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
idealCityDF


,City_ID,City Name,Latitude,Longitude,Temperature,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country
147,147,Talodi,10.6325,30.3797,76.57,76.57,76.57,34,24,5.82,SD
153,153,Riacho de Santana,-13.6092,-42.9389,72.63,72.63,72.63,63,32,4.74,BR
154,154,George Town,5.4112,100.3354,85.96,87.73,85.89,47,20,5.99,MY
189,189,Māngrol,21.1167,70.1167,73.06,73.06,73.06,60,32,8.16,IN
231,231,Tura,25.5198,90.2201,72.18,72.18,72.18,57,5,2.55,IN
250,250,Saint-Pierre,-21.3393,55.4781,82.08,82.08,82.08,61,0,5.75,RE
289,289,Gemena,3.2500,19.7667,70.79,70.79,70.79,58,25,0.83,CD
363,363,Sittwe,20.1500,92.9000,76.57,76.57,76.57,44,47,5.44,MM
391,391,Ugoofaaru,5.6667,73.0000,80.60,80.60,80.60,69,38,1.72,MV
428,428,Bhīmunipatnam,17.8833,83.4333,78.31,78.31,78.31,60,39,2.48,IN


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF.copy()



### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:

#set Base URL
base_url = "https://api.geoapify.com/v2/places"

#set initial parameters
category = "accommodation.hotel"
radius = 10000 #sets the radius to search

for index, row in hotel_df.iterrows():
    #get the latitude and longitude for city
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    #set additional parameters
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 10
    
    parameters = {
        "categories": category,
        "limit": limit,
        "filter": filters,
        "bias": bias,
        "apiKey":geoapify_key

}

    response = requests.get(base_url, params=parameters).json()

    try:
        #try to get the first hotel, and add to the dataframe
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Address"] = response["features"][0]["properties"]["address_line2"]


    except: 
        #if no hotel is found
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        hotel_df.loc[index, "Hotel Address"] = "N/A"

hotel_df.head()

,City_ID,City Name,Latitude,Longitude,Temperature,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name,Hotel Address
147,147,Talodi,10.6325,30.3797,76.57,76.57,76.57,34,24,5.82,SD,N/A,N/A
153,153,Riacho de Santana,-13.6092,-42.9389,72.63,72.63,72.63,63,32,4.74,BR,N/A,N/A
154,154,George Town,5.4112,100.3354,85.96,87.73,85.89,47,20,5.99,MY,Page 63 hostel,"Beach Street, 10300 George Town, Penang, Malaysia"
189,189,Māngrol,21.1167,70.1167,73.06,73.06,73.06,60,32,8.16,IN,N/A,N/A
231,231,Tura,25.5198,90.2201,72.18,72.18,72.18,57,5,2.55,IN,Hotel Polo Orchid,"NH 127B, - 794101, Meghalaya, India"


In [43]:
hotel_df = hotel_df[hotel_df["Hotel Address"] != "N/A"]
hotel_df

,City_ID,City Name,Latitude,Longitude,Temperature,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name,Hotel Address
154,154,George Town,5.4112,100.3354,85.96,87.73,85.89,47,20,5.99,MY,Page 63 hostel,"Beach Street, 10300 George Town, Penang, Malaysia"
231,231,Tura,25.5198,90.2201,72.18,72.18,72.18,57,5,2.55,IN,Hotel Polo Orchid,"NH 127B, - 794101, Meghalaya, India"
250,250,Saint-Pierre,-21.3393,55.4781,82.08,82.08,82.08,61,0,5.75,RE,Tropic Hotel,"Rue Auguste Babet, 97410 Saint-Pierre, France"
289,289,Gemena,3.2500,19.7667,70.79,70.79,70.79,58,25,0.83,CD,La Joie,"RN24, Bokuda-Moke, Democratic Republic of the ..."
363,363,Sittwe,20.1500,92.9000,76.57,76.57,76.57,44,47,5.44,MM,Yuzana Aung Motel 1,"Main Road, Sittwe Township, Myanmar"
428,428,Bhīmunipatnam,17.8833,83.4333,78.31,78.31,78.31,60,39,2.48,IN,Novotel,"Bheemunipatnam - Narsipatnam Road, Bheemunipat..."
462,462,Patos,-7.0244,-37.2800,78.80,78.80,78.80,63,25,6.62,BR,Hotel Zurique,"Rua Major Wanderley, Centro, Patos - PB, 58700..."
466,466,Dumka,24.2667,87.2500,72.25,72.25,72.25,26,8,1.66,IN,HOTEL ROYAL MAJESTIC,"road, Dumka - 814101, Jharkhand, India"


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
HotelPlot = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 750,
    hover_cols = ["City Name", "Country", "Hotel Name"]
)

HotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City Name,Humidity,Country,Hotel Name)